#NEED TO SAMPLE THE FILE TO DEVELOP 
awk 'BEGIN {srand()} !/^$/ { if (rand() <= .01 || FNR==1) print $0}' filename


In [184]:
!pwd


/home/dliu/lambda/unit-2/DS-Unit-2-Build-dmhliu


In [185]:
import pandas as pd
import numpy as np
import math 

pd.options.display.max_columns = 100


In [186]:
# Get Pandas Profiling Report
import pandas_profiling
#pandas_profiling.ProfileReport(df)

In [187]:
df = pd.read_csv('reduced1.csv', header=0, error_bad_lines=False, engine="python");


e 30657: ',' expected after '"'
Skipping line 30660: ',' expected after '"'
Skipping line 30683: ',' expected after '"'
Skipping line 30689: ',' expected after '"'
Skipping line 30697: ',' expected after '"'
Skipping line 30725: ',' expected after '"'
Skipping line 30748: ',' expected after '"'
Skipping line 30749: ',' expected after '"'
Skipping line 30758: ',' expected after '"'
Skipping line 30765: ',' expected after '"'
Skipping line 30783: ',' expected after '"'
Skipping line 30805: ',' expected after '"'
Skipping line 30879: ',' expected after '"'
Skipping line 30895: ',' expected after '"'
Skipping line 30900: ',' expected after '"'
Skipping line 30919: ',' expected after '"'
Skipping line 30924: ',' expected after '"'
Skipping line 30939: ',' expected after '"'
Skipping line 30995: ',' expected after '"'
Skipping line 31030: ',' expected after '"'
Skipping line 31039: ',' expected after '"'
Skipping line 31044: ',' expected after '"'
Skipping line 31045: ',' expected after '"'


In [188]:
len(df.columns)

47

In [189]:
def wrangle(df):
    X= df.copy()
    cols  = X.columns.tolist()

    dropcols = target()
    dropcols += boundary
    df.drop(labels=delete, axis=1, inplace=True)
    #do stuff
    #drop column 
    return X

In [190]:
columns = df.columns
str_cols = df.select_dtypes('object').columns
df.dtypes


CaseID                                                   object
Opened                                                   object
Closed                                                   object
Updated                                                  object
Status                                                   object
Status Notes                                             object
Responsible Agency                                       object
Category                                                 object
Request Type                                             object
Request Details                                          object
Address                                                  object
Street                                                   object
Supervisor District                                      object
Neighborhood                                             object
Police District                                          object
Latitude                                

In [191]:
delete = [s for s in df.columns.values if 'DELETE - ' in s]   #columns have delete in name
boundary = [s for s in df.columns.values if 'Boundary' in s]   #columns have boundary in name
floatcols = df.select_dtypes(include='float64').columns


In [192]:
class wrangler:
    #class vars
    dropcols = list() #list to store columns to drop
    droprows = dict() #row boolean filters

    bool_filters = dict() # list of name, expression tuples , or dict?

    encoders = dict() # colname, function to be applied to elements

    def __init__(self, data):
        self.raw_df = data.copy()  
        self.working = self.raw_df.copy()
    #methods
    def set_data(self,df):          # reset dataframe to new df
        print('set data to df', df.shape)
        self.raw_df= df.copy()
        self.working = self.raw_df.copy()
        return working
    def add_to_dropcols(self,labels):
        for l in labels:
            if l in self.dropcols:
                print('error column already in dropcols list')
                break
                return self.dropcols
        self.dropcols.extend(labels)
        return self.dropcols

    def get_dropcols(self):
        if not self.dropcols: 
            print('no columns to drop')
        return self.dropcols
    def clear_dropcols(self):
        self.dropcols=list()
        return True
    def get_nancols(self, df=None, store=False):
        if df is None: 
            df=self.working
        cols =df.columns
        nc=df[cols].isnull().sum().index.tolist()
        if store:
            self.nancols = nc
        return nc
    def add_to_droprows(self, name, expr):
        self.droprows[name] = expr 
        return self.droprows

    def get_params(self):
        pass
        #return all wrangler settings 
    def set_nanval(self):
        pass 
       # self.nanvals add self to 
    def add_bool_filter(self, label, expr):  #
        self.bool_filters[label] = expr
        return self.bool_filters
    def add_encoding(self, label, mapper):
        self.encoders[label]= mapper

#internal 
    def drop_rows_by_mask(self,df=None,labels=None):
        if df is None:
            df=self.working
        if labels is None:
            print('dropping all in droprows..')
            if self.droprows:
                labels = list(self.droprows.keys())
        mask = self.droprows[labels[0]]  #get first boolean mask
        for l in labels: 
            print('\napplying mask: ',l )
            mask = mask | self.droprows[l]   #or them all together
        df.drop(index=df[mask].index,inplace=True)
        
        return df 
    def drop_dupes(self, df=None):
        if df is None:
            df=self.working
        todropindex = df[df.duplicated()].index
        df.drop(todropindex,axis=0, inplace=True)
        return df

    def drop_columns(self,df=None):
        if df is None:
            df = self.working
        #todo check dropcols present in df.columns
        return df.drop(labels=self.dropcols, axis=1)

    def encode(self,df=None):
        if df is None: 
            df=df.working
        for k in self.encoders:
            temp[k] = temp[k].map(self.encoders[k])
        return temp
    def to_datetime(self, cols=None, df=None):
        if df is None:
            df=self.working
            print('\n working df is being changed..')
        if cols is None:
            cols=self.dt_cols
        for c in cols:
            print('converting',c,'to datetime')
            try:
                df[c] = pd.to_datetime(df[c],infer_datetime_format=True)  #inplace 
            except: 
                print('error - possible this column needs cleaning')
        return df

    order_default = [drop_rows_by_mask, 
                    drop_dupes,
                    drop_columns,
                    encode]    #list of methods in order of application

    def wrangle(self, df=None):
        if df is None:
            df =self.raw_df.copy() # start from the beginning 
            result = df
            print('will apply :', self.order_default)
            for f in self.order_default:
                print('level')
                result = f(result)
        return result
## instantiate  a wrangler and setup drops and encoding. 
## run methods on the internal working copy to iterate 
## to test entire workflow, run wrangle() with the default to process the orginal data in order defined 
## 

In [193]:
##define settings here 
test = wrangler(df)   
test.clear_dropcols()
test.add_to_dropcols(delete)
test.locationcols =['Latitude','Longitude','Point']
test.add_to_dropcols(['Point'])

missing_neighborhood = lambda x : 'missing' if math.isnan(x) else x                   #
test.encoders['Analysis Neighborhoods']= (lambda x : 'missing' if math.isnan(x) else x)

missing_to_0 = lambda x : 0 if math.isnan(x) else int(x)
test.encoders['Media URL'] = missing_to_0     

def caseid_to_int (x):    #convert good caseids to int, bad to zero for drop
    try:
        i = int(x)
    except:
        i = 0         #strings are converted to zero for dropping 
    return i

test.encoders['CaseId']= caseid_to_int

zeroids = df.CaseID.map(caseid_to_int) == 0    #mask to drop rows with no bad caseid

test.droprows['missing_caseid'] = zeroids
test.droprows.keys()


dict_keys(['missing_caseid'])

In [194]:

hoodcols = list()
for c in df.columns:
    if 'Neighborhood' in c:
        hoodcols.append(c)

dt_cols = ['Opened','Closed','Updated']
test.dt_cols = dt_cols

In [195]:

## filter some stuff 
muni = df.Category== 'MUNI Feedback'   #bus complaints we dont want

test.add_to_droprows('muni', muni);


In [196]:
test.working.shape, test.working[dt_cols].dtypes


((41684, 47), Opened     object
 Closed     object
 Updated    object
 dtype: object)

Opened     object
Closed     object
Updated    object
dtype: object

In [179]:
## run  some of the setup functionality 
test.drop_rows_by_mask()
test.to_datetime();


dropping all in droprows..

applying mask:  missing_caseid

applying mask:  muni

 working df is being changed..


AttributeError: 'wrangler' object has no attribute 'dt_cols'

In [180]:
test.order_default

[<function __main__.wrangler.drop_rows_by_mask(self, df=None, labels=None)>,
 <function __main__.wrangler.drop_dupes(self, df=None)>,
 <function __main__.wrangler.drop_columns(self, df=None)>,
 <function __main__.wrangler.encode(self, df=None)>]

In [181]:
test.add_to_droprows('muni',muni)
test.drop_rows_by_mask(temp).shape

dropping all in droprows..

applying mask:  missing_caseid

applying mask:  muni


(37537, 35)

In [182]:
pd.options.display.max_colwidth = 100 
funky = df['Opened'].str.contains('THE')
funky.isnull().sum()

nopen = df.Opened.isnull()
df[nopen]
df.head(20)


,CaseID,Opened,Closed,Updated,Status,Status Notes,Responsible Agency,Category,Request Type,Request Details,Address,Street,Supervisor District,Neighborhood,Police District,Latitude,Longitude,Point,Source,Media URL,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,DELETE - Supervisor Districts,DELETE - Fire Prevention Districts,DELETE - Current Police Districts,DELETE - Zip Codes,DELETE - Police Districts,DELETE - Neighborhoods,DELETE - Neighborhoods_from_fyvs_ahh9,DELETE - 2017 Fix It Zones,DELETE - SF Find Neighborhoods,Civic Center Harm Reduction Project Boundary,DELETE - Current Supervisor Districts,Fix It Zones as of 2017-11-06,Invest In Neighborhoods (IIN) Areas,DELETE - HSOC Zones,Fix It Zones as of 2018-02-07,"CBD, BID and GBD Boundaries as of 2017",Central Market/Tenderloin Boundary,"Areas of Vulnerability, 2016",Central Market/Tenderloin Boundary Polygon - Updated,HSOC Zones as of 2018-06-05,OWED Public Spaces,Parks Alliance CPSI (27+TL sites),Neighborhoods
0,570729,12/29/2009 06:31:54 AM,12/30/2009 10:11:25 AM,12/30/2009 10:11:25 AM,Closed,Case Transferred - PSR 353925,MUNI Work Queue,MUNI Feedback,MUNI - Services_Service_Delivery_Facilities,604_Vehicle_Appearance,Not associated with a specific address,NaN,NaN,NaN,NaN,0,0,"(0.0, 0.0)",Phone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,570341,12/28/2009 01:31:53 PM,12/30/2009 11:16:26 AM,12/30/2009 11:16:26 AM,Closed,Case Transferred - # 353946,MUNI Work Queue,MUNI Feedback,MUNI - Conduct_Inattentiveness_Negligence,201_Pass_Up_Did_Not_Wait_for_Transferee,Not associated with a specific address,NaN,NaN,NaN,NaN,0,0,"(0.0, 0.0)",Phone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,570337,12/28/2009 01:29:47 PM,12/30/2009 11:09:34 AM,12/30/2009 11:09:34 AM,Closed,Case Transferred - # 353942,MUNI Work Queue,MUNI Feedback,MUNI - Conduct_Discourteous_Insensitive_Inappropriate_Conduct,301_Discourtesy_to_Customer,Not associated with a specific address,NaN,NaN,NaN,NaN,0,0,"(0.0, 0.0)",Phone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,570026,12/28/2009 08:45:41 AM,12/30/2009 06:16:37 AM,12/30/2009 06:16:37 AM,Closed,See Notes tab for more details,DPW Ops Queue,Street and Sidewalk Cleaning,General Cleaning,Other Loose Garbage,Intersection of FOLSOM ST and RUSS ST,FOLSOM ST,6,South of Market,SOUTHERN,37.777629852295,-122.406723022461,"(37.77762985, -122.40672302)",Phone,NaN,32.0,1.0,10.0,34.0,9.0,14.0,2.0,28853.0,2.0,34.0,34.0,24.0,32.0,1.0,10.0,6.0,NaN,1.0,6.0,NaN,NaN,2.0,NaN,1.0,NaN,NaN,32.0
4,570009,12/28/2009 08:33:03 AM,12/29/2009 11:16:38 AM,12/29/2009 11:16:38 AM,Closed,See Notes tab for more details,DPW Ops Queue,Street and Sidewalk Cleaning,Bulky Items,Electronics,"901 OAK ST, SAN FRANCISCO, CA, 94117",OAK ST,5,Lower Haight,NORTHERN,37.773487091064,-122.434150695801,"(37.77348709, -122.4341507)",Phone,NaN,26.0,4.0,11.0,3.0,11.0,15.0,8.0,29492.0,5.0,9.0,9.0,NaN,26.0,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,26.0
5,569710,12/27/2009 11:13:36 AM,12/30/2009 06:16:35 AM,12/30/2009 06:16:35 AM,Closed,See Notes tab for more details,DPW Ops Queue,Street and Sidewalk Cleaning,Bulky Items,Furniture,"1101 PINE ST, SAN FRANCISCO, CA, 94109",PINE ST,3,Lower Nob Hill,CENTRAL,37.790309906006,-122.414138793945,"(37.79030991, -122.41413879)",Phone,NaN,50.0,6.0,3.0,21.0,10.0,5.0,1.0,28858.0,1.0,21.0,21.0,NaN,50.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,50.0
6,569571,12/26/2009 05:09:47 PM,12/29/2009 10:55:39 AM,12/29/2009 10:55:39 AM,Closed,Case Transferred - PSR 353805,MUNI Work Queue,MUNI Feedback,MUNI - Commendation,401_Employee_Commendation,Not associated with a specific address,NaN,NaN,NaN,NaN,0,0,"(0.0, 0.0)",Phone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [183]:
df.CaseID.map(caseid_to_int)



0          570729
1          570341
2          570337
3          570026
4          570009
           ...   
41679    11428789
41680    11450562
41681    11425262
41682    11415892
41683    11430206
Name: CaseID, Length: 41684, dtype: int64